In [13]:
from ultralytics import YOLO
import cv2
from queue import Queue
import threading
from lib.Models import Models
from lib.Dataloader import DataLoader
from enum import Enum

In [14]:
class_names = ["dog", "cow"] # Nombres de las clases a detectar con yolo
model_general = YOLO('yolo11n.pt')  # Modelo para detectar personas y vehículos
filename = "video_dogs.mp4" # Video a procesar

dog_info = {} #Almacena la información de los perros
class EnumDogList(Enum):
	FRAME = 0
	ACCURACY = 1
	COLOR = 2
	BREED = 3
	EMOTION = 4
	COUNT = 5

models = Models(dog_info)  # Modelos para el procesamiento de los caracteristicas

dataloader = DataLoader()  # Carga los datos de inteligencia y tamaño
class EnumDataLoader (Enum):
	INTELLIGENCE = 0
	SIZE = 1
	WEIGHT = 2


In [15]:
id_buffer = Queue()

pause_event = threading.Event()

pause_event.clear()

def frame_to_characteristics():
    while True:
        
        track_id = id_buffer.get()
        
        if track_id is None: pause_event.wait()
        
        pause_event.clear()
        threading.Thread(target=models.set_characteristics, args=(track_id,)).start()   

characteristics_thread = threading.Thread(target=frame_to_characteristics)
characteristics_thread.start()

In [16]:

results = model_general.track(source=filename, stream=True, verbose=False)
frame_count = 0

def set_track_id(image):
    list_characteristics = dog_info.get(track_id, [image, score, "Unknown", "Unknown", "Unknown", 0])
    list_characteristics[EnumDogList.FRAME.value] = image
    list_characteristics[EnumDogList.ACCURACY.value] = score
    dog_info[track_id] = list_characteristics

for frame_result in results:
    frame_count += 1
    frame = frame_result.orig_img
    #time.sleep(0.05)

    for box in frame_result.boxes:
        
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        score = box.conf[0].item()
        label = int(box.cls[0])
        track_id = int(box.id[0]) if box.id is not None else -1
        
        #Validación de label  dog = 16  y  Umbral de confianza
        if score>0.6 and label==16 : 
            empty_list = [0, 0, "Unknown", "Unknown", "Unknown", 0]
            last_score = dog_info.get(track_id, empty_list)[1]

            if (last_score< score):
                id_buffer.put(track_id)
                pause_event.set()
                set_track_id(frame[y1:y2, x1:x2])


        if track_id in dog_info:
            track_count = dog_info[track_id][EnumDogList.COUNT.value]
            attributes = dataloader.get_attributes(dog_info[track_id][3])
            emotion = dog_info[track_id][EnumDogList.EMOTION.value]
            size = attributes[EnumDataLoader.SIZE.value]
            intelligence = attributes[EnumDataLoader.INTELLIGENCE.value][0]
            breed = dog_info[track_id][EnumDogList.BREED.value]
            color = dog_info[track_id][EnumDogList.COLOR.value]
            
            distance = - 5
            
            cv2.rectangle(frame, (x1,  y1), (x2, y2), (0, 255, 0), 2)
            
            if breed != "Unknown":
                cv2.putText(frame,  f"Breed: {breed}", (x1, y1 + distance), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                distance -= 20
            
            if color != "Unknown":
                cv2.putText(frame,  f"Color: {color}", (x1, y1+distance), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                distance -= 20
            
            if size != "Unknown":
                cv2.putText(frame,  f"Size: {size}", (x1, y1 + distance), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                distance -= 20
            
            if emotion != "Unknown":
                cv2.putText(frame,  f"Emotion: {emotion}", (x1, y1 + distance), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                distance -= 20	
            
            if intelligence != "Unknown":
                cv2.putText(frame,  f"Intelligence: {intelligence}", (x1, y1 + distance), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                distance -= 20
            


    cv2.imshow('Detección en proceso', frame)

    # Esperar brevemente para permitir visualización, y detener si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()

El índice 'Unknown' no existe en el archivo de inteligencia
El índice 'Unknown' no existe en el archivo de atributos
El índice 'Unknown' no existe en el archivo de inteligencia
El índice 'Unknown' no existe en el archivo de atributos
El índice 'redbone' no existe en el archivo de inteligencia
El índice 'redbone' no existe en el archivo de atributos
El índice 'Unknown' no existe en el archivo de inteligencia
El índice 'Unknown' no existe en el archivo de atributos
El índice 'redbone' no existe en el archivo de inteligencia
El índice 'redbone' no existe en el archivo de atributos
El índice 'Unknown' no existe en el archivo de inteligencia
El índice 'Unknown' no existe en el archivo de atributos
El índice 'Unknown' no existe en el archivo de inteligencia
El índice 'Unknown' no existe en el archivo de atributos
El índice 'redbone' no existe en el archivo de inteligencia
El índice 'redbone' no existe en el archivo de atributos
El índice 'Unknown' no existe en el archivo de inteligencia
El í